In [7]:
import random
import numpy as np

def swap_rows(matrix, row1, row2):
    """Swaps two rows of a matrix."""
    matrix[row1], matrix[row2] = matrix[row2], matrix[row1]

def scale_row(matrix, row, scalar):
    """Scales a row of a matrix by a scalar value."""
    for i in range(len(matrix[row])):
        matrix[row][i] *= scalar

def add_multiple_of_row_to_row(matrix, source_row, target_row, scalar):
    """Adds a multiple of one row to another row in a matrix."""
    for i in range(len(matrix[target_row])):
        matrix[target_row][i] += scalar * matrix[source_row][i]

def ref(matrix):
    """Converts a matrix to row echelon form (REF) with pivot normalization."""
    num_rows = len(matrix)
    num_cols = len(matrix[0])
    lead = 0

    for r in range(num_rows):
        if lead >= num_cols:
            break

        i = r
        while matrix[i][lead] == 0:
            i += 1
            if i == num_rows:
                i = r
                lead += 1
                if lead == num_cols:
                    break

        swap_rows(matrix, i, r)  # Pivoting

        # Normalize the pivot element to 1
        pivot_value = matrix[r][lead]
        # Divide each element in the pivot row by the pivot value
        for i in range(len(matrix[r])):  # Iterate through each element in the row
            matrix[r][i] /= pivot_value

        for j in range(r + 1, num_rows):
            if matrix[j][lead] != 0:
                add_multiple_of_row_to_row(matrix, r, j, -matrix[j][lead])  # Elimination (no division needed now)

        lead += 1

def rref(matrix):
    """Converts a matrix to reduced row echelon form (RREF)."""
    ref(matrix)  # Perform REF first
    num_rows = len(matrix)
    num_cols = len(matrix[0])

    for r in range(num_rows - 1, -1, -1):  # Iterate from bottom to top
        for j in range(r):
            if matrix[j][r] != 0:
                add_multiple_of_row_to_row(matrix, r, j, -matrix[j][r])  # Eliminate elements above the pivot

        if matrix[r][r] != 0:
            scale_row(matrix, r, 1 / matrix[r][r])  # Scale the pivot to 1

def identify_pivot_columns(matrix):
    """Identifies pivot and non-pivot columns in a matrix."""
    pivot_cols = []
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] == 1 and all(matrix[k][j] == 0 for k in range(i)):  # Pivot condition
                pivot_cols.append(j)
                break
    non_pivot_cols = [i for i in range(len(matrix[0])) if i not in pivot_cols]
    return pivot_cols, non_pivot_cols

def find_particular_solution(rref_matrix):
    """Finds the particular solution to Ax = b from the RREF matrix."""
    solution = [0] * (len(rref_matrix[0]) - 1)  # Initialize solution vector
    print("solution ",solution)
    for i in range(len(rref_matrix)):
        if 1 in rref_matrix[i]:  # Check if there's a leading 1
            solution[rref_matrix[i].index(1)] = rref_matrix[i][-1]
        else:
            # Handle cases without a leading 1 (e.g., dependent variables)
            print("Row without a leading 1:", rref_matrix[i])
            # You might want to assign a symbolic value or raise a specific exception here
    return solution

def get_homogeneous_solutions(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    pivot_columns, nonpivot_columns = identify_pivot_columns(matrix)

    solutions = []

    for col in nonpivot_columns:
        solution = np.zeros((cols - 1, 1))
        if col < len(solution):  # Check if col is within bounds
            solution[col] = 1

        for row, pivot_col in enumerate(pivot_columns):
            if pivot_col < len(solution):  # Check if pivot_col is within bounds
                solution[pivot_col, 0] = -matrix[row][col]

        solutions.append(solution.flatten().tolist())

    return solutions


    
while True:
    try:
        m = int(input("Enter the number of rows (m) (must be less than the number of columns): "))
        n = int(input("Enter the number of columns (n): "))
        if m > n:
            print("Invalid input. Rows (m) must be less than columns (n). Please try again.")
            continue
        break
    except ValueError:
        print("Invalid input. Please enter integers for rows and columns.")

while True:
    choice = input("Choose an option for matrix A:\n1. Generate a random matrix\n2. Enter matrix elements manually\nEnter your choice (1 or 2): ")
    if choice in ["1", "2"]:
        break
    else:
        print("Invalid choice. Please enter 1 or 2.")

if choice == "1":
    A = [[random.randint(1, 10) for _ in range(n)] for _ in range(m)]  # Generate random matrix A
else:
    A = []
    for i in range(m):
        row = []
        for j in range(n):
            while True:
                try:
                    value = int(input(f"Enter value for element A({i+1},{j+1}): "))
                    row.append(value)
                    break
                except ValueError:
                    print("Invalid input. Please enter an integer.")
        A.append(row)

# Input for vector b
b = []
for i in range(m):
    while True:
        try:
            value = int(input(f"Enter value for element b{i+1}: "))
            b.append(value)
            break
        except ValueError:
            print("Invalid input. Please enter an integer.")

# Construct augmented matrix
augmented_matrix = [[A[i][j] for j in range(n)] + [b[i]] for i in range(m)]

print("Matrix A:")
for row in A:
    print([int(x) for x in row])

print("Vector b:",b)

print("Augmented matrix:")
for row in augmented_matrix:
    print([int(x) for x in row])

# Perform REF
ref(augmented_matrix)
print("\nREF:")
for row in augmented_matrix:
    print([int(x) for x in row])

# Perform RREF
rref(augmented_matrix)
print("\nRREF:")
for row in augmented_matrix:
    print([int(x) for x in row])
    

# Identify pivot and non-pivot columns
pivot_cols, non_pivot_cols = identify_pivot_columns(augmented_matrix)
print("\nPivot columns:", pivot_cols)
print("Non-pivot columns:", non_pivot_cols)

# Find particular solution
particular_solution = find_particular_solution(augmented_matrix)
print("\nParticular solution:", particular_solution)

# Find solutions to Ax = 0
# Get solutions to Ax = 0
homogeneous_solutions = get_homogeneous_solutions(augmented_matrix)
print("\nSolutions to Ax = 0:")
for solution in homogeneous_solutions:
    print(solution)

Matrix A:
[2, 6, 9, 5, 6]
[8, 7, 6, 7, 8]
[3, 9, 10, 7, 2]
[8, 2, 4, 2, 7]
Vector b: [-1, 2, 3, 4]
Augmented matrix:
[2, 6, 9, 5, 6, -1]
[8, 7, 6, 7, 8, 2]
[3, 9, 10, 7, 2, 3]
[8, 2, 4, 2, 7, 4]

REF:
[1, 3, 4, 2, 3, 0]
[0, 1, 1, 0, 0, 0]
[0, 0, 1, 0, 2, -1]
[0, 0, 0, 1, 4, -4]

RREF:
[1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, -4, 4]
[0, 0, 1, 0, 1, 0]
[0, 0, 0, 1, 4, -4]

Pivot columns: [0, 1, 2, 3]
Non-pivot columns: [4, 5]
solution  [0, 0, 0, 0, 0]

Particular solution: [0.8749999999999982, 4.0624999999999964, -0.6875000000000011, -4.187499999999993, 0]

Solutions to Ax = 0:
[-0.2890624999999982, 4.957031249999995, -1.3398437500000013, -4.621093749999991, 1.0]
[-0.8749999999999982, -4.0624999999999964, 0.6875000000000011, 4.187499999999993, 0.0]


Vishal 

Matrix A:
[2, 6, 9, 5, 6]
[8, 7, 6, 7, 8]
[3, 9, 10, 7, 2]
[8, 2, 4, 2, 7]
Vector b: [-1, 2, 3, 4]
Augmented matrix:
[2, 6, 9, 5, 6, -1]
[8, 7, 6, 7, 8, 2]
[3, 9, 10, 7, 2, 3]
[8, 2, 4, 2, 7, 4]

REF:
[1, 3, 4, 2, 3, 0]
[0, 1, 1, 0, 0, 0]
[0, 0, 1, 0, 2, -1]
[0, 0, 0, 1, 4, -4]

RREF:
[1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, -4, 4]
[0, 0, 1, 0, 1, 0]
[0, 0, 0, 1, 4, -4]

Pivot columns: [0, 1, 2, 3]
Non-pivot columns: [4, 5]
solution  [0, 0, 0, 0, 0]

Particular solution: [0.8749999999999982, 4.0624999999999964, -0.6875000000000011, -4.187499999999993, 0]

Solutions to Ax = 0:
[-0.2890624999999982, 4.957031249999995, -1.3398437500000013, -4.621093749999991, 1.0]
[-0.8749999999999982, -4.0624999999999964, 0.6875000000000011, 4.187499999999993, 0.0]


Amit bhai 

Invalid input. Please enter an integer.
Matrix A:
[2, 7, 2, 6, 7]
[2, 3, 3, 5, 10]
[8, 5, 4, 1, 8]
[9, 8, 9, 1, 7]
Vector b: [5, 4, 1, 2]
Augmented matrix:
[2, 7, 2, 6, 7, 5]
[2, 3, 3, 5, 10, 4]
[8, 5, 4, 1, 8, 1]
[9, 8, 9, 1, 7, 2]

REF:
[1, 3, 1, 3, 3, 2]
[0, 1, 0, 0, 0, 0]
[0, 0, 1, 1, 3, 1]
[0, 0, 0, 1, 2, 0]

RREF:
[1, 0, 0, 0, 1, 0]
[0, 1, 0, 0, -1, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 2, 0]

Pivot columns: [0, 1, 2, 3]
Non-pivot columns: [4, 5]
solution  [0, 0, 0, 0, 0]

Particular solution: [-0.1093544137022402, 0.11725955204216085, 0.15151515151515182, 0.6824769433465084, 0]

Solutions to Ax = 0:
[-1.372859025032937, 1.1949934123847163, -0.24242424242424354, -2.022397891963109, 1.0]
[0.1093544137022402, -0.11725955204216085, -0.15151515151515182, -0.6824769433465084, 0.0]


Anshuman 

Matrix A:
[9, 5, 4, 1, 1]
[10, 1, 10, 3, 7]
[4, 6, 9, 1, 9]
[8, 6, 4, 7, 1]
Vector b: [1, 2, 3, 4]
Augmented matrix:
[9, 5, 4, 1, 1, 1]
[10, 1, 10, 3, 7, 2]
[4, 6, 9, 1, 9, 3]
[8, 6, 4, 7, 1, 4]

REF:
[1, 0, 0, 0, 0, 0]
[0, 1, -1, 0, -1, 0]
[0, 0, 1, 0, 1, 0]
[0, 0, 0, 1, 0, 0]

RREF:
[1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0]
[0, 0, 0, 1, 0, 0]

Pivot columns: [0, 1, 2, 3]
Non-pivot columns: [4, 5]
solution  [0, 0, 0, 0, 0]

Particular solution: [-0.15392124959323136, 0.22974292222583792, 0.20013016596160105, 0.43605597136348845, 0]

Solutions to Ax = 0:
[0.4334526521314675, -0.07614708753660904, -1.1513179303612104, 0.08493328994467945, 1.0]
[0.15392124959323136, -0.22974292222583792, -0.20013016596160105, -0.43605597136348845, 0.0]
